In [2]:
# Modeling using TITLE instead of content

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import collections, re
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
import re
import tensorflow as tf

/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [78]:
#read data
data = pd.read_json('2.News_With_Labels.json')
data.head()

,createdate,articles
0,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [79]:
#convert to df
df = pd.DataFrame(data)
df.head()

,createdate,articles
0,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [80]:
clear

In [81]:
stock_df = pd.DataFrame([])

for i in df['articles']:
    stock_df = stock_df.append(pd.DataFrame({'title': i['title'], 'label': i['label'], 'category': i['category']}, index=[0]), ignore_index=True)

print(f"Number of words in raw data: {stock_df['title'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

Number of words in raw data: 48384


,title,label,category
0,How an A.I. ‘Cat-and-Mouse Game’ Generates Bel...,Buy,Technology
1,Activists’ Guide to 2018: The Best Defense Is ...,Buy,Business
2,"Logan Paul, YouTube Star, Says Posting Video o...",Buy,Business
3,How to Manage Your Career,Buy,Business Day
4,Adjusting Twitter’s Sensitivities,Buy,Business


In [82]:
# Get names of indexes for which column category is Technology
indexNames = stock_df[ stock_df['category'] == 'Technology'].index
 
stock_df.drop(indexNames , inplace=True)

In [83]:
# # Get names of indexes for which column category is Business Day
# indexNames2 = stock_df[ stock_df['category'] == 'Business Day'].index
 
# stock_df.drop(indexNames2 , inplace=True)

In [84]:
# Get names of indexes for which column category is Politics
indexNames3 = stock_df[ stock_df['category'] == 'Politics'].index
 
stock_df.drop(indexNames3 , inplace=True)

In [85]:
stock_df.head()

,title,label,category
1,Activists’ Guide to 2018: The Best Defense Is ...,Buy,Business
2,"Logan Paul, YouTube Star, Says Posting Video o...",Buy,Business
3,How to Manage Your Career,Buy,Business Day
4,Adjusting Twitter’s Sensitivities,Buy,Business
5,TV Content Wars: Too Many Characters Chasing N...,Buy,Business


In [86]:
#keep a count of how many you delete
bad_rows = []
for i in range(0,stock_df.shape[0]):
    #if the number of characters is below 100
    if len(stock_df['title'].iloc[i])<100:
        bad_rows.append(i)

In [50]:
import nltk
nltk.download('stopwords')

#clean up the text
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

#function to clean the text
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text
    
#apply function to each row in the dataframe
stock_df['title'] = stock_df['title'].apply(clean_text)
#number of words after clean up
print(f"Number of words AFTER clean up: {stock_df['title'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mollyperlich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of words AFTER clean up: 116


,title,label,category
0,ai catandmouse game generates believable fake ...,Buy,Technology
42,asked ces 2018 answered,Sell,Technology
273,ratio establishes twitter,Buy,Technology
279,social media gives women voice,Buy,Technology
535,selfdriving uber killed pedestrian arizona,Hold,Technology


In [51]:
content_ = []
for item in stock_df["title"]:
    if (item in content_):
        continue
    else:
        content_.append(item)

In [52]:
from sklearn.model_selection import train_test_split

X = stock_df.title
y = stock_df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [53]:
bagsofwords = [ collections.Counter(re.findall(r'\w+', txt))
            for txt in content_]
bagsofwords[0]

Counter({'ai': 1,
         'catandmouse': 1,
         'game': 1,
         'generates': 1,
         'believable': 1,
         'fake': 1,
         'photos': 1})

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content_) 
print(X.toarray())
print(vectorizer.get_feature_names())

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['2018', 'access', 'across', 'ai', 'alexa', 'answered', 'answers', 'apples', 'arizona', 'asked', 'believable', 'best', 'breach', 'caregiver', 'carry', 'catandmouse', 'ces', 'consider', 'conversation', 'costs', 'could', 'data', 'deceptive', 'deep', 'device', 'disinformation', 'distracted', 'edition', 'establishes', 'evolving', 'facebook', 'fake', 'followers', 'friends', 'game', 'gave', 'generates', 'gift', 'gives', 'google', 'guide', 'hands', 'hate', 'holiday', 'home', 'house', 'india', 'information', 'internet', 'iphones', 'killed', 'leads', 'lost', 'made', 'make', 'makers', 'many', 'marriott', 'media', 'meet', 'minaj', 'mobs', 'much', 'murder', 'new', 'nicki', 'oceans', 'online', 'others', 'pedestrian', 'photos', 'post', 'protect', 'purge', 'questions', 'quiz', 'ratio', 'robot', 'selfdriving', 'selfies', 'siri', 'smart', 'social', 'speech', 'spot', 'spread', 'superin

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents


# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(content_)
# summarize
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# encode document
#vector = vectorizer.transform([text[0]])
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [56]:
vector = vectorizer.transform([content_[1]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 104)
[[0.45445063 0.         0.         0.         0.         0.51428741
  0.         0.         0.         0.51428741 0.         0.
  0.         0.         0.         0.         0.51428741 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        

# Bag-of-Words with Keras
The Keras Python library for deep learning also provides tools for encoding text using the bag-of words-model in the Tokenizer class.
As above, the encoder must be trained on source documents and then can be used to encode training data, test data and any other data in the future. The API also has the benefit of performing basic tokenization prior to encoding the words.


In [57]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer(num_words=2)
texts = content_
tk.fit_on_texts(texts)
# # create the tokenizer
# t = Tokenizer()
# # fit the tokenizer on the documents
# t.fit_on_texts(content_)
# # summarize what was learned
# # print(t.word_counts)
# # print(t.document_count)
# # print(t.word_index)
# # print(t.word_docs)
# # integer encode documents

In [58]:
tk.texts_to_sequences(texts)

[[],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [1],
 []]

In [59]:
tk.word_index

{'facebook': 1,
 'ai': 2,
 '2018': 3,
 'twitter': 4,
 'uber': 5,
 'gave': 6,
 'robot': 7,
 'vs': 8,
 'meet': 9,
 'us': 10,
 'catandmouse': 11,
 'game': 12,
 'generates': 13,
 'believable': 14,
 'fake': 15,
 'photos': 16,
 'asked': 17,
 'ces': 18,
 'answered': 19,
 'ratio': 20,
 'establishes': 21,
 'social': 22,
 'media': 23,
 'gives': 24,
 'women': 25,
 'voice': 26,
 'selfdriving': 27,
 'killed': 28,
 'pedestrian': 29,
 'arizona': 30,
 'weekly': 31,
 'edition': 32,
 'device': 33,
 'makers': 34,
 'deep': 35,
 'access': 36,
 'data': 37,
 'users': 38,
 'friends': 39,
 'internet': 40,
 'travels': 41,
 'across': 42,
 'oceans': 43,
 'could': 44,
 'weaponized': 45,
 'spread': 46,
 'disinformation': 47,
 'purge': 48,
 'many': 49,
 'followers': 50,
 'trump': 51,
 'nicki': 52,
 'minaj': 53,
 'others': 54,
 'lost': 55,
 'hands': 56,
 'evolving': 57,
 'alexa': 58,
 'siri': 59,
 'google': 60,
 'carry': 61,
 'conversation': 62,
 'best': 63,
 'spot': 64,
 'deceptive': 65,
 'post': 66,
 'whatsapp': 67

In [60]:
encoded_docs = tk.texts_to_matrix(texts, mode='count')
print(encoded_docs)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]]


In [61]:
print(len(encoded_docs))
# print(len(text[i]))

23


# Classification Models

In [62]:
train_data_features = vectorizer.fit_transform(content_)

In [63]:
#logisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.4
             precision    recall  f1-score   support

        Buy       0.50      0.67      0.57         3
       Hold       0.00      0.00      0.00         2
       Sell       0.00      0.00      0.00         0

avg / total       0.30      0.40      0.34         5



/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [64]:
#RandomForest

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

r_forests = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=100)),
               ])
r_forests.fit(X_train, y_train)

y_pred = r_forests.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.6
             precision    recall  f1-score   support

        Buy       0.60      1.00      0.75         3
       Hold       0.00      0.00      0.00         2

avg / total       0.36      0.60      0.45         5



/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
#Keras 

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(stock_df) * .7)
train_posts = stock_df['title'][:train_size]
train_tags = stock_df['label'][:train_size]

test_posts = stock_df['title'][train_size:]
test_tags = stock_df['label'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 4

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 14 samples, validate on 2 samples
Epoch 1/4
14/14 [==============================] - 0s 19ms/step - loss: 1.0988 - acc: 0.2857 - val_loss: 1.1154 - val_acc: 0.0000e+00
Epoch 2/4
14/14 [==============================] - 0s 472us/step - loss: 1.0630 - acc: 0.3571 - val_loss: 1.0995 - val_acc: 0.0000e+00
Epoch 3/4
14/14 [==============================] - 0s 511us/step - loss: 1.0097 - acc: 0.7857 - val_loss: 1.0835 - val_acc: 0.0000e+00
Epoch 4/4
14/14 [==============================] - 0s 492us/step - loss: 0.9763 - acc: 0.8571 - val_loss: 1.0680 - val_acc: 0.0000e+00


In [66]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 176us/step
Test accuracy: 0.2857142984867096
